In [4]:
import pandas as pd 
import numpy as np
import time
import math
import random
import os
from collections import Counter
import copy
import pickle
import os 
import csv
import re

In [5]:
datasets="cm2.5k"
window_size=250

In [8]:
def generate(datasets,window_size):
    input_file="datasets/"+datasets+"/"+datasets+".csv"
    # window_size=100
    data=pd.read_csv(input_file)
    try:
        data.columns = ["CaseID", "Activity", "TYPE", "Timestamp","amount"]
    except:
        data.columns = ["CaseID","Activity","Type","Timestamp"]
    data.fillna(1,inplace=True)
    listofactivity=data["Activity"].unique().tolist()
    cont_trace = data['CaseID'].value_counts(dropna=False)
    max_trace=max(cont_trace)
    # group by activity, resource and timestamp by caseid
    act = data.groupby('CaseID', sort=False).agg({'Activity': lambda x: list(x)})
    temp = data.groupby('CaseID', sort=False).agg({'Timestamp': lambda x: list(x)})
    caseid=data.groupby("CaseID",sort=False).agg({"CaseID":lambda x:list(x)})
    # if not os.path.exists("datasets/"+datasets+"/"+datasets+"_"+str(window_size)):
    #     os.mkdir("datasets/"+datasets+"/"+datasets+"_"+str(window_size))
    # else:
    #     return
    # ttime="2799-02-2015:57:07"
    # date_format_str = '%Y-%m-%d%H:%M:%S'
    # time.strptime(ttime, date_format_str)
    def time_format(ttime):
        ttime=" ".join(ttime.split("T"))
        ttime=ttime.split(".")[0]
        try:
            date_format_str = '%Y-%m-%d %H:%M:%S'
            conversion = time.strptime(ttime, date_format_str)
        except:
            date_format_str = '%Y/%m/%d %H:%M:%S'
            conversion = time.strptime(ttime, date_format_str)
        return conversion
    time_differences=[1e9,-1]
    time_lasts=[1e9,-1]
    time_remaining=[1e9,-1]
    def prefix_padding_time_normalize(prefixs):
        for i in range(len(prefixs)):
            for j in range(len(prefixs[i])):
                prefixs[i][j][2]=prefixs[i][j][2]/time_differences[1]
        for i in range(len(prefixs)):
            trace=prefixs[i]
            if len(trace)<max_trace:
                num=max_trace-len(trace)
                for j in range(num):
                    prefixs[i].insert(0,[0 for m in range(24+7+2+1+1)])
        return prefixs        

    def time_normalize(labels):
        for i in range(len(labels)):
            labels[i][1]=labels[i][1]/time_remaining[1]
        return labels
    def prefix_generating(sub_log):
        act_list=sub_log[0]
        temp_list=sub_log[1]
        id_list=sub_log[2]
        prefixs=[]
        labels=[]
        lengths=[]
        for act_trace,temp_trace,id_trace in zip(act_list,temp_list,id_list):
        # print(act_trace)      
            for i in range(1,len(act_trace)):
                act_prefix=act_trace[:i]
                temp_prefix=temp_trace[:i]
                id=id_trace[0]
                tempdata=[]
                for j in range(i):
                    act_vector_j=np.array([id]+[act_prefix[j]])
                    first_day=time_format(temp_trace[0])
                    previous_day=time_format(temp_trace[j-1 if j!=0 else 0])
                    current_day= time_format(temp_trace[j])
                    next_day=time_format(temp_trace[i])
                    last_day=time_format(temp_trace[-1])
                    # temp_vector=["持续时间","前一个时间差","距离午夜时间"]
                    timelast=0 if j==0 else (time.mktime(current_day)-time.mktime(first_day))
                    timedifference = 0 if j==0 else (time.mktime(current_day)-time.mktime(previous_day))
                    timeremaining= -1*(time.mktime(current_day)-time.mktime(last_day))
                    nexteventtime= -1*(time.mktime(current_day)-time.mktime(next_day))

                    time_differences[0]=min(time_differences[0],timedifference)
                    time_differences[1]=max(time_differences[1],timedifference)

                    time_lasts[0]=min(time_lasts[0],timelast)
                    time_lasts[1]=max(time_lasts[1],timelast)

                    time_remaining[0]=min(time_remaining[0],timeremaining)
                    time_remaining[1]=max(time_remaining[1],timeremaining)

                    dayhour=np.array([1 if i==current_day.tm_hour else 0 for i in range(24)],dtype="float")   
                    wday=np.array([1 if i==current_day.tm_wday else 0 for i in range(7)],dtype="float")
                    time_vector_j = np.concatenate((np.array([timelast]),np.array([timedifference]),dayhour,wday),axis=0)
                    # time_vector_j = np.array([timelast,timedifference,current_day.tm_hour])
                    tempdata.append(np.concatenate((act_vector_j,time_vector_j),axis=0))
                templabel=[act_trace[i],timeremaining]
                prefixs.append(tempdata)
                labels.append(templabel)
                lengths.append(i)
        # prefixs=prefix_padding(prefixs)
        # labels=time_normalize(labels)
        return prefixs,labels,lengths

    act,temp=act.loc[:,"Activity"].values,temp.loc[:,"Timestamp"].values
    caseid=caseid.loc[:,"CaseID"].values
    dataset_length=np.size(act,0)
    num_sublogs=math.ceil(dataset_length/window_size)
    datas=[]
    for i in range(num_sublogs):
        start=i*window_size
        end=min((i+1)*window_size,dataset_length)
        result=prefix_generating([act[start:end],temp[start:end],caseid[start:end]])
        datas.append(result)


    for i in range(len(datas)):
        prefixs=datas[i][0]
        labels=datas[i][1]
        lengths=datas[i][2]
        prefixs=prefix_padding_time_normalize(datas[i][0])
        labels=time_normalize(datas[i][1])
        if not os.path.exists("datasets/"+datasets+"/"+datasets+"_"+str(window_size)+"/sub_logs_"+str(i)):
            os.mkdir("datasets/"+datasets+"/"+datasets+"_"+str(window_size)+"/sub_logs_"+str(i))
        pickle.dump(prefixs,open("datasets/"+datasets+"/"+datasets+"_"+str(window_size)+"/sub_logs_"+str(i)+"/"+"data.pkl","wb"))
        pickle.dump(labels,open("datasets/"+datasets+"/"+datasets+"_"+str(window_size)+"/sub_logs_"+str(i)+"/"+"label.pkl","wb"))
        pickle.dump(lengths,open("datasets/"+datasets+"/"+datasets+"_"+str(window_size)+"/sub_logs_"+str(i)+"/"+"length.pkl","wb"))



        

In [9]:
generate("Base_cp10k",100)

In [ ]:
father_dir="datasets"
sub_dirs = [sub_dir for sub_dir in os.listdir(father_dir) if (os.path.isdir(os.path.join(father_dir, sub_dir)) and "Base" in sub_dir)]
for sub_dir in sub_dirs:
    generate(sub_dir,100)

In [ ]:
datasets=["cm2.5k","cm5k","cm7.5k","cm10k"]
window_sizes=[25,50,75,100]
for i in range(len(datasets)):
    for j in range(len(window_sizes)):
        generate(datasets[i],window_sizes[j])